# POC 5

## Inference

Let's walk through deploying a DCN model in Sagemaker!

In [ ]:
#!pip install easydict

In [ ]:
%pylab inline
import os
import boto3
import time
import io
from matplotlib.pyplot import imshow, imread
import subprocess
from skimage.transform import resize as skresize
from PIL import Image
import requests
import cv2
import random
import numpy as np
import json
import sagemaker
from sagemaker.predictor import StringDeserializer
from sagemaker.predictor import RealTimePredictor, json_deserializer
import image_pb2 as impb
import ast
from IPython.display import clear_output
from dcn.lib.utils.show_boxes import show_boxes


sess = sagemaker.Session() # can use LocalSession() to run container locally
bucket = 'privisaa-bucket-virginia' #sess.default_bucket() # can replace with our own bucket 
role = sagemaker.session.get_execution_role()
s3 = boto3.client('s3')

#!cp /home/ec2-user/SageMaker/SageMaker-Inference-Advanced/n-labs/11-manual-model-load/dcn/model/rfcn_dcn_coco-0000.params dcn/model/

Lets take a look at our inference script

In [ ]:
!pygmentize inference.py

In order to deploy our model to an endpoint, we need to put our weights into gzip format

In [ ]:
!(cd dcn/model/ && tar -czvf /tmp/model-rfcn.tar.gz *)

In [ ]:
s3.upload_file('/tmp/model-rfcn.tar.gz', bucket, 'super_models/model-rfcn.tar.gz')
!aws s3 ls s3://privisaa-bucket-virginia/super_models/   

Now we create our container we will use for DCN inference.

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=mxnet-serving-160-gpu-py2

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

#docker build -t chazarey-mxnet-serving:1.6.0-gpu-py3 -f docker/1.6.0/py3/Dockerfile.gpu .

docker build -t ${algorithm_name} -f Dockerfile.gpu .

docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Now that we have built our container, we can deploy our endpoint. This process usually takes some time.

In [ ]:
%%time

from sagemaker.mxnet import MXNetModel

model_data="s3://privisaa-bucket-virginia/super_models/model-rfcn.tar.gz"

model = MXNetModel(
    model_data=model_data,
    role=role,
    image="209419068016.dkr.ecr.us-east-1.amazonaws.com/chazarey-mxnet-serving-160-gpu-py2",
    entry_point="inference.py",
    py_version='py2',
    framework_version='1.6.0',
    enable_cloudwatch_metrics=True
)

# for deploying the endpoint locally for testing we can use an instance_type of local
#predictor = model.deploy(instance_type="local_gpu", initial_instance_count=1)
#predictor = model.deploy(instance_type="local", initial_instance_count=1)

predictor = model.deploy(instance_type='ml.g4dn.2xlarge', initial_instance_count=1)

In [ ]:
num_classes = 81
classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
           'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
           'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
           'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
           'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
           'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
           'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse',
           'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
           'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

Now that we have deployed our endpoint, let's test it out!
We are going to package our image into a protobuf object and then send it to our endpoint for classification. 
Right now it takes approximately 60ms end to end 

In [ ]:
%%time

with open(impath, 'rb') as f:
    payload = f.read()

image_packet = impb.PBImage()
image_packet.image_data = payload

predictor.serializer = None
predictor.deserializer = StringDeserializer()
predictor.accept = None
predictor.content_type = 'application/octet-stream'

response = predictor.predict(image_packet.SerializeToString())          


Let's look at our image prior to classification

In [ ]:
impath = '/home/ec2-user/SageMaker/SageMaker-Inference-Advanced/n-labs/11-manual-model-load/dcn/demo/COCO_test2015_000000000891.jpg'

img = imread(impath)
imshow(img)

In [ ]:
resp_list = []
for e in eval(response)['array']:
    resp_list.append(np.array(e))

In [ ]:
#im = cv2.imread('./dcn/demo/' + im_name)
im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
show_boxes(im, resp_list, classes, 1)

In [ ]:
%time response = predictor.predict(image_packet.SerializeToString())            
